In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy

In [3]:
from importlib import reload
import lib2
from IPython.display import clear_output
from lib import data_management as dm
from time import sleep
import drivers
from lib2.IQPulseSequence import *
from drivers.KeysightAWG import *
from drivers.IQAWG import *
warnings.filterwarnings("ignore",".*GUI is implemented.*")
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [4]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_res_mixer_to_-10_2", -10))
ro_cal = calibrations.get(frozenset(dict(lo_power=7, ssb_power=-45, lo_frequency=7.5351e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_qub_mixer_to_-10", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-10, lo_frequency=6.163e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=1).items()))
q_pb = PulseBuilder(q_cal)

In [5]:
ro_awg = KeysightAWG("AWG3")

In [6]:
q_awg = KeysightAWG("AWG2")

In [7]:
ro_iqawg = IQAWG(AWGChannel(ro_awg, 1), AWGChannel(ro_awg, 2))
q_iqawg = IQAWG(AWGChannel(q_awg, 1), AWGChannel(q_awg, 2))

In [358]:
from drivers.Keysight_DSOX2014 import *

In [359]:
dso = Keysight_DSOX2014("DSO2014")

______________

### Resonators scan areas

In [30]:
scan_areas = [(7529945247.2623634, 7579945247.2623634),
  (7541330816.5408268, 7591330816.5408268),
  (7552773888.6944351, 7602773888.6944351),
  (7564561978.0989046, 7614561978.0989046),
  (7574164958.2479124, 7624164958.2479124)]

In [31]:
scan_areas

[(7529945247.262363, 7579945247.262363),
 (7541330816.540827, 7591330816.540827),
 (7552773888.694435, 7602773888.694435),
 (7564561978.098905, 7614561978.098905),
 (7574164958.247912, 7624164958.247912)]

# Spectroscopy

## Single-tone spectroscopy

In [8]:
#reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [10]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(800,2).build())

In [9]:
STS = SingleToneSpectroscopy("III-anticrossing", "Xmons_Nb_bandage", vna = 'vna1', src = 'yok3')
vna_parameters = {"bandwidth":500, "freq_limits":(7.52e9, 7.55e9), "nop":1001, "sweep_type": "LIN", "power":7, "averages":1}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(1.2e-3, 1.5e-3, 31)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);
result.visualize();

The device vna1 is detected as PNA-L1
The device yok3 is detected as GS210_3
Started at:  2017-08-11 14:56:45.434610
Time left: 0 h 0 m 38.62 s, [current: 1.31e-03]: , average cycle time: 2.03 s       

In [27]:
result.visualize();

In [13]:
result.save()

## Two-tone spectroscopy

In [10]:
# reload(lib2.Measurement)
#reload(lib2.TwoToneSpectroscopyBase)
#reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [12]:
q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(800,2).build())
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(800,2).build())

In [16]:
ro_awg._visainstrument.write(":ARM:SOUR1 IMM")

(16, <StatusCode.success: 0>)

#### Qubit spectrum

In [21]:
TTS = FluxTwoToneSpectroscopy("I-two-tone", "Xmons_Nb_bandage", vna_name="vna1", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":50, "freq_limits":(7.525e9, 7.545678e9), "nop":20, "sweep_type":"LIN", "power":-12, "averages":1}
mw_src_parameters = {"power":0}
mw_src_frequencies = linspace(5.8e9, 6.3e9, 101)
currents = linspace(1.2e-3, 1.5e-3, 31)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545678000.0)                    at qubit's sweet spot (1.35 mA)
Detected frequency is 7.53511 GHz, at 12.24 mU and -18.04 degrees


In [ ]:
result = TTS.launch()

Started at:  2017-08-11 15:03:52.973064
Time left: 0 h 20 m 27.97 s, parameter value: 1.210e-03, average cycle time: 0.41 s          

In [14]:
result.visualize();

In [186]:
result.save()

##### TTS result processing and fitting

In [72]:
angles = np.angle(result.get_data()['data'])
curs = result.get_data()['Current [A]']
freqs = result.get_data()['frequency']

In [76]:
TTS_pts = []
for i,ang_slice in enumerate(angles[:]):
    TTS_pts.append(freqs[np.argmax(ang_slice)])

In [237]:
def f_spectr(x, x0, A, delt):
    return (-1*np.sqrt((A*(x-x0))**2 + (delt)**2)) + 2*delt

In [243]:
plt.plot(curs, f_spectr(np.array(curs), 1.35e-3, 1e14, 6e9));

In [248]:
popt, pcov = scipy.optimize.curve_fit(f_spectr, curs[1:-2], np.array(TTS_pts[1:-2]), p0=[1.35e-3, 1e14, 6.15e9]);
plt.plot(curs[1:-2], np.array(TTS_pts[1:-2]), 'o', curs[1:-2], f_spectr(curs[1:-2], *popt));
plt.grid(True)

___________________

In [ ]:
result._name = "5.5GHz-two-tone-w-loop_-0.5mA-and-coil"

In [ ]:
TTS._mw_src.set_output_state("OFF")

#### Linewidth from excitation power

In [283]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(1000, 1).build())
q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(1000, 1).build())

In [58]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmons_Nb_bandage", vna_name='vna1', mw_src_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":200, "freq_limits":(7.52e9, 7.55e9), "nop":50, "power":7, "sweep_type":"CW","averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(6.14e9, 6.18e9,401)
mw_src_powers = linspace(7, 7.1, 2)
current = 1.36e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7520000000.0, 7550000000.0)                    at current of 1.36 mA
Detected frequency is 7.53511 GHz, at 4.53 mU and -123.33 degrees


In [59]:
result = PTS.launch()

Started at:  2017-08-08 19:45:43.804415
Time left: 0 h 3 m 26.8 s, parameter value: 7.000e+00, average cycle time: 0.28 s           

In [336]:
result.set_plot_range(min_abs=0.19, max_abs=0.21, min_phas=65, max_phas=85)

In [320]:
result._plot_limits_fixed = True

In [51]:
result.visualize();

In [50]:
TTS._actual_devices.keys()

dict_keys(['vna1', 'mxg', 'yok3', 'awg3', 'awg2'])

In [ ]:
TTS._construct_fixed_parameters()

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [338]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [95]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

AttributeError: 'KeysightAWG' object has no attribute 'output_pulse_sequence'

In [ ]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-35, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.40e9, 7.46e9, 100)
vna_powers = linspace(-30, 0, 31)
current = 2.355e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

In [ ]:
result = ASTS.launch()

In [87]:
result.visualize()

(<matplotlib.figure.Figure at 0x11962518>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x0000000011B92A20>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000001235A898>], dtype=object),
 (<matplotlib.axes._axes.Axes at 0x123bcd30>,
  <matplotlib.axes._axes.Axes at 0x12422160>))

In [86]:
result.save()

In [274]:
TTS.close_devs("mxg")

__________________________

# Time-domain measurements

### Check mixer calibration

In [22]:
q_lo = TTS._mw_src
q_lo.set_output_state("ON")
q_lo.set_frequency(6.14e9)
q_lo.set_power(10)

In [55]:
q_lo.visa_instr.close()

In [142]:
TTS._vna.set_nop(1)
TTS._vna.set_power(7)
TTS._vna.set_center(7.53511e9)
TTS._vna.sweep_continuous()

In [18]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [19]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(200,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(200,5).build())

In [28]:
q_cal.get_radiation_parameters()

{'if_frequency': 50000000.0,
 'lo_frequency': 6210000000.0,
 'lo_power': 7,
 'ssb_power': -22,
 'waveform_resolution': 0.1}

### Set qubit frequency

In [23]:
TTS._current_src.set_current(1.33e-3)

True

In [23]:
q_freq

6164000000.0

In [27]:
q_freq = 6.164e9

In [ ]:
q_freq = q_freq+0.03e6
q_freq

### Rabi 

In [29]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [106]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(900, 1).add_zero_pulse(100).build())
q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(900, 1).add_zero_pulse(100).build())

In [34]:
DRO = DispersiveRabiOscillations("I-rabi-recal", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":3, "freq_limits":(7.525e9, 7.54501e9), "nop":3, "sweep_type":"LIN", "averages":1}
exc_frequency = q_freq
excitation_durations = linspace(0, 1500, 501)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":1000, "repetition_period":8000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545010000.0)                    
Detected frequency is 7.53506 GHz, at 3.19 mU and -139.71 degrees


In [35]:
result = DRO.launch()

Started at:  2017-08-11 13:15:56.939507
Time left: 0 h 5 m 26.93 s, [excitation_duration: 8.10e+02]: , average cycle time: 1.42 s        

Traceback (most recent call last):
  File "C:\Users\PAINMASTER\Measurement\lib2\Measurement.py", line 196, in launch
    plt.pause(self._plot_update_interval)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py", line 298, in pause
    canvas.draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 176, in draw
    FigureCanvasAgg.draw(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 464, in draw
    self.figure.draw(self.renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 63, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1143, in draw
    renderer, self, dsu, self.suppressComposite)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py", line 139, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\ProgramData\Ana

In [214]:
q_iqawg.get_calibration().get_optimization_results()

({'dc_offsets': array([ 0.00217761, -0.02069942]),
  'dc_offsets_open': None,
  'if_amplitudes': array([ 0.4911197 ,  0.46191546]),
  'if_offsets': array([ 0.00108721, -0.01972392]),
  'if_phase': array([-1.60931904])},
 {'dc': -72.808212280273438,
  'if': [-10.028465270996094, -36.78864288330078, -86.7591781616211]})

In [36]:
result.visualize();

In [37]:
result.save()

In [84]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

69.272230575451886

In [20]:
pi_pulse_duration = 58.66

### Decay

In [237]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [256]:
DD = DispersiveDecay("I-decay", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg', )
vna_parameters = {"bandwidth":20, "freq_limits":(7525e6, 7545e6), "nop":40, "sweep_type":"CW", "averages":5}
q_frequency = q_freq
readout_delays = linspace(0, 5000,101)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":700, 
                            "repetition_period":7000, "pi_pulse_duration":pi_pulse_duration}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DD.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 0.73 mU and -65.96 degrees


In [257]:
dd_result = DD.launch()

Started at:  2017-08-09 20:55:19.016021
Time left: 0 h 0 m 0.0 s, [readout_delay: 5.00e+03]: , average cycle time: 9.38 s          

In [292]:
dd_result.visualize();

In [259]:
dd_result.save()

### Ramsey

In [26]:
# reload(lib2.IQPulseSequence)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [27]:
DR = DispersiveRamsey("I-ramsey", "Xmons_Nb_bandage", vna_name="vna1", ro_awg=ro_iqawg, 
                                                                                  q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7525e6, 7545.5e6), "nop":10, "averages":2, "sweep_type":"LIN"}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq+5e6
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":800, 
                            "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545500000.0)                    
Detected frequency is 7.53510 GHz, at 0.87 mU and -56.44 degrees


In [28]:
dr_result = DR.launch()

Started at:  2017-08-10 10:29:46.450430
Time left: 0 h 3 m 42.84 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.35 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 3 m 33.22 s, [ramsey_delay: 1.62e+02]: , average cycle time: 2.34 s       

In [305]:
dr_result.visualize();

In [306]:
dr_result.save()

### Echo

In [80]:
# reload(lib2.VNATimeResolvedMeasurement)
# from lib2.VNATimeResolvedMeasurement import *
# reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [89]:
DHN = DispersiveHahnEcho("I-hahn-echo", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7525e6, 7545e6), "nop":20, "sweep_type": "LIN", "averages":2}
q_frequency = q_freq
echo_delays = linspace(10, 5000, 101)
sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DHN.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, sequence_parameters)
DHN.set_swept_parameters(echo_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 0.85 mU and -71.64 degrees


In [90]:
dhn_result = DHN.launch()

Started at:  2017-08-10 14:18:41.850050
Time left: 0 h 0 m 0.0 s, [echo_delay: 5.00e+03]: , average cycle time: 4.13 s         

In [87]:
dhn_result.visualize();

In [88]:
dhn_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-hahn-echo").visualize()

### Ramsey fringes

In [103]:
# reload(lib2.Measurement)
# reload(drivers.KeysightAWG)
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveRamseyFringes)
from lib2.DispersiveRamseyFringes import *

In [124]:
DRF = DispersiveRamseyFringes("I-ramsey-fringes", "Xmons_Nb_bandage",
                                      vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7524e6, 7544e6), "nop":1, "sweep_type":"CW", "averages":3}
q_frequency =6.1575e9
excitation_freqs = linspace(q_frequency-5e6, q_frequency+5e6, 101)
ramsey_delays = linspace(10, 5000, 100)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRF.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRF.set_swept_parameters(ramsey_delays, excitation_freqs)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7524000000.0, 7544000000.0)                    
Detected frequency is 7.53506 GHz, at 0.81 mU and -67.34 degrees


In [125]:
drf_result = DRF.launch()

Started at:  2017-08-10 18:04:32.275090
Time left: 0 h 3 m 40.02 s, [ramsey_delay: 4.70e+03, excitation_frequency: 6.16e+09]: , average cycle time: 0.33 s        

In [ ]:
drf_result.visualize()

In [126]:
drf_result.save()

### Rabi shevrons

In [71]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.DispersiveRabiChevrons)
from lib2.DispersiveRabiChevrons import *

In [74]:
DRC = DispersiveRabiChevrons("I-rabi-shevrons", "Xmons_Nb_bandage",
                                      vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7525e6, 7545e6), "nop":4, "averages":2, "sweep_type": "LIN"}
q_frequency = q_freq
excitation_freqs = linspace(q_frequency-5e6, q_frequency+5e6, 41)
rabi_delays = linspace(10, 3000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRC.set_swept_parameters(rabi_delays, excitation_freqs)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 0.89 mU and -66.79 degrees


In [75]:
drc_result = DRC.launch()

Started at:  2017-08-10 11:17:10.793118
Time left: 0 h 0 m 0.0 s, [excitation_duration: 3.00e+03, excitation_frequency: 6.17e+09]: , average cycle time: 0.78 s          

In [78]:
drc_result.visualize();

In [77]:
drc_result.save()

### Dispersive shift measurement

In [ ]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.TimeResolvedDispersiveShiftSpectroscopy)
from lib2.TimeResolvedDispersiveShiftSpectroscopy import *

In [ ]:
DSS = TimeResolvedDispersiveShiftSpectroscopy("VI-rabi-dispersive-shift-exact-q-freq", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":100, "averages":2}
exc_frequency = 7.89919e9
excitation_durations = linspace(1, 3000, 100)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DSS.set_fixed_parameters(vna_parameters, q_frequency, ro_awg_params, q_awg_params, sequence_parameters)
DSS.set_swept_parameters(excitation_durations)

In [ ]:
dss_result = DSS.launch()

In [ ]:
dss_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-rabi-dispersive-shift").visualize();

In [ ]:
dss_result.visualize();

______________________________________________

# Pulse optimization

## Amplified phase error

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveAPE)
from lib2.DispersiveAPE import *

### Rectangular pulses

In [ ]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":2}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000}
    APE_sequence_parameters.update({"half_pi_pulse_duration":pi_pulse_duration/2, 
                                    "pseudo_I_pulses_count":i, "modulating_window":'rectangular',
                                    "excitation_amplitude":1, "padding":5})
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()

In [ ]:
dape_result = DAPE.launch()

In [ ]:
dape_result.visualize();

In [ ]:
dape_result.save()

### Gaussian pulses

#### Rabi

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersivePiPulseAmplitudeCalibration)
from lib2.DispersivePiPulseAmplitudeCalibration import *

In [ ]:
DPPAC = DispersivePiPulseAmplitudeCalibration("VI-pi-amp-cal", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = q_freq
excitation_amps = linspace(0, 3, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000,
                            "repetition_period":8000, "excitation_duration":50, "modulating_window":"gaussian"}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DPPAC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, sequence_parameters)
DPPAC.set_swept_parameters(excitation_amps)

In [ ]:
dppac_result = DPPAC.launch()

In [ ]:
dppac_result.visualize();

In [ ]:
pi_pulse_amplitude = 2.28
pi_pulse_amplitude

#### Ramsey 

In [ ]:
from lib2.DispersiveRamsey import *

In [119]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                                          ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
ramsey_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                              "excitation_amplitude":pi_pulse_amplitude/2, "modulating_window":"gaussian",
                              "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, ramsey_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

TypeError: __init__() got an unexpected keyword argument 'ro_awg_name'

In [ ]:
drg_result = DR.launch()

In [ ]:
drg_result.visualize();

#### APE

In [ ]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-gaussian-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":3}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "padding":5,
                            "half_pi_pulse_duration":50, "modulating_window":"gaussian",
                            "excitation_amplitude":pi_pulse_amplitude/2, "pseudo_I_pulses_count":i}
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()
    clear_output()

In [ ]:
dape_result.visualize()

---------------------------

In [ ]:
reload(lib2.IQPulseSequence)
from lib2.IQPulseSequence import *

In [ ]:
pb = PulseBuilder(q_cal)

In [ ]:
APE_sequence_parameters["ramsey_angle"]=0
APE_sequence_parameters["pseudo_I_pulses_count"] = 1
APE_sequence_parameters["excitation_amplitude"] = 1

In [ ]:
APE_sequence_parameters

In [ ]:
PulseBuilder.build_dispersive_APE_sequences(pb, ro_pb, APE_sequence_parameters)[0].plot()
pb.add_zero_pulse(400).add_sine_pulse(65).add_sine_pulse(50).add_zero_pulse(15).build().plot()
plt.xlim(300,600)

In [ ]:
# pb.add_zero_pulse(10).add_sine_pulse(30).add_zero_pulse(10).build().plot()
pb.add_zero_pulse(10).add_sine_pulse(50, window=("gaussian", {"amplitude":10})).add_zero_pulse(10).build().plot()
plt.grid()

In [ ]:
rect = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1).add_zero_pulse(1000).build()

In [ ]:
gaus = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1, window="gaussian").add_zero_pulse(1000).build()

In [ ]:
rect.plot()
gaus.plot()

In [ ]:
q_awg.output_pulse_sequence(gaus)

In [ ]:
exp(-1/8)

In [ ]:
X = linspace(-1,1)

In [ ]:
plot(X, exp(-(3*X)**2/2**2))

In [ ]:
from resonator_tools import circuit

In [ ]:
circuit.notch_port()